### DATA COLLECTION

In [3]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import copy
import csv
import os
from lib.rtls_slave import pixel_calculate, aoa_main, sort_data
import json


ENVIRONMENT = 'random'

# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:yaw>/<signed_int:roll>/<signed_int:pitch>', methods=['GET'])
def get_angle(yaw,roll,pitch):
    print(f"YAW: {yaw}, ROLL: {roll}, PITCH: {pitch}")
    global aoa
    global aoabefore
    
    print("Plane Angle: " + str(roll))
    aoa_result = aoa_main()
    input_letter = input("If you want to add this data to list type 'y', otherwise press Enter")
    if input_letter!='y':
        return jsonify({'state':'Try Again'})
    lev.append(roll)
    aoabefore = copy.deepcopy(aoa)
    aoa = aoa_result

    with open(f"./data/aoa_data/{ENVIRONMENT}_{roll}.json",'w') as f:
        json.dump(aoa_result, f)
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    # ANGLE SORTING TECHNIQUES
    aoabefore_ = sort_data(aoabefore,aoa_bias = -35)
    aoa_ = sort_data(aoa,aoa_bias = -35)

    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    set_aoa = set(aoa_)
    set_aoabefore = set(aoabefore_)

    for slave in set_aoa.intersection(set_aoabefore):
        f = open(f"./results/pixels/{slave.replace(':','_')}.csv",'w',newline="")
        f.truncate()
        writer = csv.writer(f)
        
        data_before = aoabefore_[slave]
        data_after = aoa_[slave]

        for i in range(len(data_before[0])):
            for j in range(len(data_after[0])):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], data_before[0][i], data_after[0][j])
                count = data_before[1][i]*data_after[1][j]
                if temp1 != -1:
                    tux = (temp1, temp2,count)
                    writer.writerow(tux)

        f.close()
                
        
        f.close()

    
    return jsonify({'state': 'success'})

In [5]:
ENVIRONMENT = 'random'
app.run(host='0.0.0.0',port = 5000, debug=False)
# My Charger: 80:6F:B0:EE:AD:32 : Water Bottle.
# Ishrath Charger:  80:6F:B0:EE:AA:13 : Tea Bottle.

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.19.217.212:5000
Press CTRL+C to quit


YAW: -68, ROLL: 4, PITCH: 178
Plane Angle: 4
=== ERROR: No one of the nodes identified as RTLS MASTER ===
Try to stop AOA result parsing thread
AOA Stopped
Done
{}


10.19.190.195 - - [05/May/2023 15:41:33] "GET /send/-68/4/178 HTTP/1.1" 200 -


YAW: -1, ROLL: 0, PITCH: -108
Plane Angle: 0
=== ERROR: No one of the nodes identified as RTLS MASTER ===
Try to stop AOA result parsing thread
AOA Stopped
Done
{}


10.19.190.195 - - [05/May/2023 15:43:01] "GET /send/-1/0/-108 HTTP/1.1" 200 -


### POST POINT CALCULATION

In [15]:
# Some Manual Twiking Required
post_calculation('calibration_set1_ii',68,74,aoa_bias = -50)

Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[-5.80571429 -5.27238095 -4.73904762 -4.20571429] 
	AoA After:[-4.81333333 -4.28       -3.74666667 -3.21333333]
	Center: (2316.763856802069, 1849.4252781336704)



'POINTS CALCULATION SUCCESSFUL'

In [16]:
from lib.rtls_slave import post_calculation


with open("./data/aoa_data/calculations.txt", "r") as file:
    for line in file:
        ctnt = line.split('_')
        CASE = '_'.join(ctnt[:-2])
        L1 = int(ctnt[-2])
        L2 = int(ctnt[-1])
        print(f"CASE: {CASE}")
        post_calculation(CASE,L1,L2,aoa_bias = -50)

CASE: calibration_set1_i
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[-11.78494624 -11.2516129  -10.71827957 -10.18494624] 
	AoA After:[-11.73365854 -11.25365854 -10.77365854 -10.29365854  -9.81365854
  -9.33365854]
	Center: (2982.302048950748, 1824.690260992856)

CASE: calibration_set1_ii
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[-5.80571429 -5.27238095 -4.73904762 -4.20571429] 
	AoA After:[-4.81333333 -4.28       -3.74666667 -3.21333333]
	Center: (2316.763856802069, 1849.4252781336704)

CASE: calibration_set1_iii
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[28.36336996 28.82051282 29.27765568 29.73479853 30.19194139 30.64908425
 31.10622711 31.56336996] 
	AoA After:[33.24648649 33.72648649 34.20648649 34.68648649 35.16648649 35.64648649]
	Center: (-209.69310279408091, -633.8178150874869)

CASE: calibration_set1_iv
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[24.20540541 24.73873874 25.27207207 25.8

f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:293: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:293: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))


### VISUALIZING THE ANGLE

In [ ]:
from lib.rtls_slave import visualize_aoa_spread

CASE = 'three_down_indoor_i'
level = 74
visualize_aoa_spread(CASE, level)

In [ ]:
from lib.rtls_slave import visualize_aoa_turn_spread

CASE = 'two_top_indoor'
level1 = 68
level2 = 76
vis_bias = 0

visualize_aoa_turn_spread(CASE, level1, level2, vis_bias)

### RAW DATA VISUALIZATION

In [ ]:
import matplotlib.pyplot as plt
import json

def visualize_aoa_spread(CASE, level, num_bins = 100):

    with open(f'./data/aoa_data/{CASE}_{level}.json', 'r') as f:
        aoa_data = json.load(f)

    # Create a figure and axis object
    fig, ax = plt.subplots()

    # Set the x and y labels and title
    ax.set_xlabel("Angle of Arrival")
    ax.set_ylabel("# of Occurance")
    ax.set_title("Histogram of Angle of Arrival")

    # Set the number of bins in the histogram

    # Loop over each slave and plot its angle of arrival data as a histogram
    for slave, data in aoa_data.items():
        ax.hist(data, bins=num_bins, alpha=0.5, label=slave)

    # Add a legend to the plot
    ax.legend()

    # Show the plot
    plt.show()

    if len(aoa_data.keys())>1:
        # Set up the figure with subplots
        fig, axs = plt.subplots(nrows=1, ncols=len(aoa_data.keys()), figsize=(10, 5))

        # Iterate over each slave and plot a histogram of its angle of arrival data
        for i, slave in enumerate(aoa_data.keys()):
            axs[i].hist(aoa_data[slave], bins=num_bins,color='lightblue')  # adjust bins as needed
            axs[i].set_title(slave)
            axs[i].set_xlabel("Angle of Arrival")
            axs[i].set_ylabel("# of Occurance")

        # Adjust the spacing between subplots and display the figure
        plt.tight_layout()

        plt.show()
        
        return "SUCESS"
    

visualize_aoa_spread('one_top_indoor',70,100)